In [1]:
import pandas as pd
import numpy as np

In [2]:
meta_df = pd.read_csv('C:/Users/juwieczo/DataspellProjects/meisd_project/data/MEISD_text.csv')

In [3]:
# For the multilabel classification we use:
columns = ['Utterances', 'dialog_ids', 'intensity', 'intensity2', 'intensity3']
meta_dfs = meta_df[columns].copy()

In [4]:
meta_dfs.loc[:, 'intensity'] = pd.to_numeric(meta_dfs['intensity'], errors='coerce')
meta_dfs.loc[:, 'intensity2'] = pd.to_numeric(meta_dfs['intensity2'], errors='coerce')
meta_dfs.loc[:, 'intensity3'] = pd.to_numeric(meta_dfs['intensity3'], errors='coerce')


#Zamień wartości na liczby całkowite
meta_dfs['intensity'] = pd.to_numeric(meta_dfs['intensity'], errors='coerce').fillna(0)
meta_dfs['intensity2'] = pd.to_numeric(meta_dfs['intensity2'], errors='coerce').fillna(0)
meta_dfs['intensity3'] = pd.to_numeric(meta_dfs['intensity3'], errors='coerce').fillna(0)

#Zamień wartości zawierające tylko białe znaki lub '`', 'neu', 'po' na NaN
meta_dfs['intensity'] = meta_dfs['intensity'].replace(r'^\s*$', np.nan, regex=True)
meta_dfs['intensity'] = meta_dfs['intensity'].replace(['`', 'neu', 'po'], np.nan)
meta_dfs['intensity2'] = meta_dfs['intensity2'].replace(r'^\s*$', np.nan, regex=True)
meta_dfs['intensity2'] = meta_dfs['intensity2'].replace(['`', 'neu', 'po'], np.nan)
meta_dfs['intensity3'] = meta_dfs['intensity3'].replace(r'^\s*$', np.nan, regex=True)
meta_dfs['intensity3'] = meta_dfs['intensity3'].replace(['`', 'neu', 'po'], np.nan)

#Użyj forward fill, aby uzupełnić brakujące wartości poprzedzającą wartością
meta_dfs['intensity'] = meta_dfs['intensity'].ffill()
meta_dfs['intensity2'] = meta_dfs['intensity2'].ffill()
meta_dfs['intensity3'] = meta_dfs['intensity3'].ffill()

#Usuń znaki niebędące cyframi (np. '`') za pomocą wyrażeń regularnych
meta_dfs['intensity'] = meta_dfs['intensity'].replace(r'\D', '', regex=True).astype(int)  # Usuwa wszystko, co nie jest cyfrą
meta_dfs['intensity2'] = meta_dfs['intensity2'].replace(r'\D', '', regex=True).astype(int)
meta_dfs['intensity3'] = meta_dfs['intensity3'].replace(r'\D', '', regex=True).astype(int)

In [5]:
first_25_data = []
last_25_data = []

def process_group(group):
    num_rows = len(group)
    quarter_size = max(1, num_rows // 4)

    # First 25%
    first_25 = group.iloc[:quarter_size]
    last_25 = group.iloc[-quarter_size:]

    # Maksymalna intensywność
    first_25_max_intensity = first_25['intensity'].max()
    last_25_max_intensity = last_25['intensity'].max()

    # Łączenie wypowiedzi już tutaj
    first_utterances = " ".join(first_25['Utterances'])
    last_utterances = " ".join(last_25['Utterances'])

    first_25_data.append(pd.Series({
        'dialog_ids': group['dialog_ids'].iloc[0],
        'Utterances': first_utterances,
        'max_intensity': first_25_max_intensity
    }))

    last_25_data.append(pd.Series({
        'dialog_ids': group['dialog_ids'].iloc[0],
        'Utterances': last_utterances,
        'max_intensity': last_25_max_intensity
    }))

meta_dfs.groupby('dialog_ids').apply(process_group)

first_25_df = pd.DataFrame(first_25_data)
last_25_df = pd.DataFrame(last_25_data)


C:\Users\juwieczo\AppData\Local\Temp\ipykernel_3788\3186010028.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  meta_dfs.groupby('dialog_ids').apply(process_group)


In [7]:
first_25_df

,dialog_ids,Utterances,max_intensity
0,1,look around you say hello to your competition ...,0
1,2,"i'm george o'malley uh, we met at the mixer. y...",2
2,3,seattle is surrounded by water on three sides ...,2
3,4,yes no other reason? just a favor for an old p...,2
4,5,if he doesn't respond to these tests in the ne...,2
...,...,...,...
1119,1121,i was thinking exactly the same thing no. crim...,2
1120,1122,"just to be clear, he backed down an insurance ...",3
1121,1123,"hey, michael, did you fall into a koi pond? i ...",2
1122,1124,"hey, boss, did you find nemo? i can name pixar...",2


In [8]:
last_25_df.to_csv('max_first_25_intensity.csv', index=False)

In [ ]:
last_25_df.head()

In [ ]:
# Tworzenie nowej kolumny bazującej na intensity
last_25_df['binary_intensity'] = (last_25_df['max_intensity'] == 2).astype(int)

In [ ]:
last_25_df